# Reading charm quark from LHE file, Fragmentation, and Conversion 

## Load Libraries 
At first, I load all the libraries we will need later on.

In [1]:
import time 
import numpy as np
import os
import pythia8
from skhep.math.vectors import LorentzVector, Vector3D

### Function to write events in common format 

In [2]:
def count_events(filename):
    nevent, nevents = 0 , 0
    with open(filename) as f:
        for line in f:
            if line[:-1]=="<event>": nevents+=1
            words = [elt.strip() for elt in line.split( )]
            if words[0]=="<event>": nevent+=1
    print (nevent, nevents)
            
def round_y_and_pt(y0, pt0 , ybins, ptbins, dy, dpt):
    iy = np.digitize(abs(y0),ybins)
    ipt = np.digitize(pt0,ptbins)
    y = ybins[iy]-dy/2.
    pt = ptbins[ipt]-dpt/2.
    return y, pt

def format_spectra(results, label):
               
    # setup grid
    pids = [411, 421, 431, 4122, -411, -421, -431, -4122]
    ymin,ymax,ny = 0,9,18
    ptmin,ptmax,npt = 0,30,120
    dy,dpt = (ymax-ymin)/float(ny),(ptmax-ptmin)/float(npt)
    grid = {}
    yvals = np.linspace(ymin+dy/2., ymax-dy/2., ny)
    ptvals = np.linspace(ptmin+dpt/2., ptmax-dpt/2., npt)
    ybins = np.linspace(ymin, ymax, ny+1)
    ptbins = np.linspace(ptmin, ptmax, npt+1)
    for pid in pids:
        thisgrid = {}
        for y in yvals:
            for pt in ptvals:
                thisgrid[round(pt, 3),round(y, 3)] = 0
        grid[pid]=thisgrid
            
    # split up results
    data, total_xs, nEvent = results
            
    # read data
    for pid, yh, pth in data:
        if pth < ptmax and abs(yh)<9:
            y,pt = round_y_and_pt(abs(yh), pth, ybins, ptbins, dy, dpt)
            grid[pid][pt, y] +=1
        
    # normalization and write
    total_xs *= 0.5 # for forward/backward
    dirname = "files/"+label+"/"
    if not os.path.exists(dirname): os.mkdir(dirname)
    for pid in pids:
        f = open(dirname+"/"+label+"_"+str(pid)+".txt", "w")
        for pt in ptvals:
            for y in yvals:
                weight = grid[pid][round(pt, 3),round(y, 3)] * total_xs / nEvent / dpt / dy
                f.write(str(round(y, 3))+" "+str(round(pt, 3))+" "+str(weight)+"\n" )
        f.close()

## Run Events through Pythia

In [59]:
def process_events_pythia(filename, nrepeat=1):
        
    #get total cross section 
    with open(filename) as f:
        found_first_event=False
        for line in f:
            if found_first_event: 
                total_xs =  float(line.split( )[2])   
                break
            if line[:-1]=="<event>": 
                found_first_event=True
                
    #run pythia multiple times
    data, nEvent = [], 0
    for i in range(nrepeat):
        
        # Initialize pythia 
        pythia = pythia8.Pythia();
        pythia.readString("Random:setSeed = on");
        pythia.readString("Random:seed = 0")
        pythia.readString("411:maydecay=off");
        pythia.readString("421:maydecay=off");
        pythia.readString("431:maydecay=off");
        pythia.readString("4122:maydecay=off");
        pythia.readString("111:maydecay=off");

        # Initialize Les Houches Event File run.
        pythia.readString("Beams:frameType = 4");
        pythia.readString("Beams:LHEF =" + filename);
        pythia.init();

        # Begin event loop
        iEvent = 0
        while (True):

            # logging info
            #if iEvent>=10000: break;
            iEvent+=1
            nEvent+=1
            if (iEvent+1)% 10000 == 0: print ("processed "+str(iEvent+1)+" Events")

            # generate next event and stop if reach end
            pythia.next();
            if pythia.info.atEndOfFile(): break
                #print (iEvent)
                #print ("End of file:", pythia.info.atEndOfFile())
                #print (pythia.event)
                

            # analyze events
            for iparticle, particle in enumerate(pythia.event):
                if particle.status()<0: continue
                if abs(particle.id()) in [411, 421, 431, 4122]: 
                    px, py = particle.px(), particle.py()
                    pz, en = particle.pz(), particle.e()
                    pt, y  = np.sqrt(px**2+py**2), 0.5*np.log((en+pz)/(en-pz))
                    data.append([particle.id(), y, pt])     
        print ("found "+str(iEvent)+" events") 
    
    return data, total_xs, nEvent

## Use Fragmentation Function

In [4]:
def process_events_ff(filename, nrepeat=1, frame="lab", scale="p"):
        
    # read charm quarks
    events = []  
    with open(filename) as f:
        found_event=False
        for line in f:
            if found_event: 
                words = [elt.strip() for elt in line.split( )]
                if len(words)==0 or words[0][0]=="#": continue
                if len(words)==6: total_xs = float(words[2])   
                if len(words)==13 and words[0] in ["4","-4"]: 
                    px, py = float(words[6]), float(words[7])
                    pz, en = float(words[8]), float(words[9])
                    event.append(LorentzVector(px, py, pz, en))
            if line[:-1]=="<event>": 
                found_event=True
                event = []
            if line[:-1]=="</event>": 
                found_event=False
                events.append(event)
            #if len(events)>=10000: break
    
    # initialize fragmentation
    mcharm=1.5
    epsilon=0.05
    pids = [411  , 421  , 431  , 4122 ]
    masses = {411: 1.86962, 421: 1.86486, 431: 1.96849, 4122: 2.28646}
    ffractions = [0.246, 0.610, 0.082, 0.062]
    ffractions =  np.array(ffractions) / sum(ffractions)
    zvals = np.linspace(0.005,0.995,100)
    ffvals = [ (1-1/z-epsilon/(1-z))**(-2)/z for z in zvals]
    ffvals =  np.array(ffvals) / sum(ffvals) 
    
    # charm 
    data=[]
    for irepeat in range(nrepeat):
        for event in events: 

            system = event[0]+event[1]
            if frame=="lab": 
                quarks = [event[0], event[1]]
            if frame=="cm" : 
                q1 = event[0].boost(system.boostvector)
                q2 = event[1].boost(system.boostvector)
                quarks = [q1, q2]
 
            for iparticle, particle in enumerate(quarks):

                # select hadronization channel
                pid = np.random.choice(pids, size=1, p=ffractions)[0]
                mass = masses[pid]

                # select z=ph/pq and x=Eh/Eq
                e, p = particle.e, particle.p
                px, py, pz = particle.px, particle.py, particle.pz
                z = np.random.choice(zvals, size=1, p=ffvals)[0]
                #x = np.sqrt((z**2 * p**2 + mass**2)/(e**2))

                # hadron momentum 
                #hadron0 =  LorentzVector(px*z, py*z, pz*z, e*x)
                hadron =  LorentzVector()
                if scale=="p": 
                    hadron.setpxpypzm(z*px, z*py, z*pz, mass)
                if scale=="e":
                    if z*e<mass: hadron.setpxpypzm(0, 0, 0, mass)
                    else: 
                        x = np.sqrt(z**2*e**2-mass**2)/p
                        hadron.setpxpypzm(x*px, x*py, x*pz, mass)
                if frame=="cm": hadron=hadron.boost(-1.*system.boostvector)

                # save
                data.append([(-1)**(2-iparticle)*pid, hadron.rapidity, hadron.pt])           
    return data, total_xs, len(events)*nrepeat

## Run stuff

In [5]:
filename="/Users/felixkling/Dropbox/Events_for_Felix/eventfile_13TeV_gg2ccb_hyb_MRW-MMHT2014nlo.lhe"

In [6]:
start_time = time.time()
results = process_events_pythia(filename, nrepeat=1)
format_spectra(results, "13TeV_KT_RAFAL-hyb-MRW-P8")
print ("- finished after", round(time.time() - start_time,1), "seconds")

/Users/felixkling/anaconda/envs/rootenv/lib/python2.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log


processed 10000 Events
processed 20000 Events
processed 30000 Events
processed 40000 Events
processed 50000 Events
processed 60000 Events
60539
('End of file:', True)

 --------  PYTHIA Event Listing  (complete event)  ---------------------------------------------------------------------------------
 
    no         id  name            status     mothers   daughters     colours      p_x        p_y        p_z         e          m 
                                   Charge sum:  0.000           Momentum sum:      0.000      0.000      0.000      0.000      0.000

 --------  End PYTHIA Event Listing  -----------------------------------------------------------------------------------------------

found 60539 events
('- finished after', 1911.7, 'seconds')


In [5]:
60539*8+40


484352

In [15]:
start_time = time.time()
results = process_events_ff(filename, nrepeat=5, frame="lab")
format_spectra(results, "13TeV_KT_RAFAL-hyb-MRW-FFLab")
print ("- finished after", round(time.time() - start_time,1), "seconds")

('load', '/Users/felixkling/Dropbox/Events_for_Felix/eventfile_13TeV_gg2ccb_hyb_MRW-MMHT2014nlo.lhe', '- finished after', 2318.6, 'seconds')


In [14]:
start_time = time.time()
results = process_events_ff(filename, nrepeat=5, frame="cm")
format_spectra(results, "13TeV_KT_RAFAL-hyb-MRW-FFCM")
print ("- finished after", round(time.time() - start_time,1), "seconds")

('load', '/Users/felixkling/Dropbox/Events_for_Felix/eventfile_13TeV_gg2ccb_hyb_MRW-MMHT2014nlo.lhe', '- finished after', 3528.2, 'seconds')


In [13]:
start_time = time.time()
results = process_events_ff(filename, nrepeat=5, frame="lab", scale="e")
format_spectra(results, "13TeV_KT_RAFAL-hyb-MRW-FFLabE")
print ("- finished after", round(time.time() - start_time,1), "seconds")

('load', '/Users/felixkling/Dropbox/Events_for_Felix/eventfile_13TeV_gg2ccb_hyb_MRW-MMHT2014nlo.lhe', '- finished after', 10702.8, 'seconds')


In [19]:
count_events(filename)

(1790523, 1790523)


# Generating with Pythia, Different Fragmentations

### Load Libraries

In [3]:
import time 
import numpy as np
import os
import pythia8
from skhep.math.vectors import LorentzVector, Vector3D

### Function to write events in common format 

In [4]:
def count_events(filename):
    nevent, nevents = 0 , 0
    with open(filename) as f:
        for line in f:
            if line[:-1]=="<event>": nevents+=1
            words = [elt.strip() for elt in line.split( )]
            if words[0]=="<event>": nevent+=1
    print (nevent, nevents)
            
def round_y_and_pt(y0, pt0 , ybins, ptbins, dy, dpt):
    iy = np.digitize(abs(y0),ybins)
    ipt = np.digitize(pt0,ptbins)
    y = ybins[iy]-dy/2.
    pt = ptbins[ipt]-dpt/2.
    return y, pt

def format_spectra(results, label):
               
    # setup grid
    pids = [411, 421, 431, 4122, -411, -421, -431, -4122]
    ymin,ymax,ny = 0,9,18
    ptmin,ptmax,npt = 0,30,120
    dy,dpt = (ymax-ymin)/float(ny),(ptmax-ptmin)/float(npt)
    grid = {}
    yvals = np.linspace(ymin+dy/2., ymax-dy/2., ny)
    ptvals = np.linspace(ptmin+dpt/2., ptmax-dpt/2., npt)
    ybins = np.linspace(ymin, ymax, ny+1)
    ptbins = np.linspace(ptmin, ptmax, npt+1)
    for pid in pids:
        thisgrid = {}
        for y in yvals:
            for pt in ptvals:
                thisgrid[round(pt, 3),round(y, 3)] = 0
        grid[pid]=thisgrid
            
    # split up results
    data, total_xs, nEvent = results
            
    # read data
    for pid, yh, pth in data:
        if pth < ptmax and abs(yh)<9:
            y,pt = round_y_and_pt(abs(yh), pth, ybins, ptbins, dy, dpt)
            grid[pid][pt, y] +=1
        
    # normalization and write
    total_xs *= 0.5 # for forward/backward
    dirname = "files/"+label+"/"
    if not os.path.exists(dirname): os.mkdir(dirname)
    for pid in pids:
        f = open(dirname+"/"+label+"_"+str(pid)+".txt", "w")
        for pt in ptvals:
            for y in yvals:
                weight = grid[pid][round(pt, 3),round(y, 3)] * total_xs / nEvent / dpt / dy
                f.write(str(round(y, 3))+" "+str(round(pt, 3))+" "+str(weight)+"\n" )
        f.close()

### Fragmentation Functions

In [27]:
# initialize fragmentation
epsilon=0.05
pids = [411  , 421  , 431  , 4122 ]
masses = {411: 1.86962, 421: 1.86486, 431: 1.96849, 4122: 2.28646}
ffractions = [0.246, 0.610, 0.082, 0.062]
ffractions =  np.array(ffractions) / sum(ffractions)
zvals = np.linspace(0.005,0.995,100)
ffvals = [ (1-1/z-epsilon/(1-z))**(-2)/z for z in zvals]
ffvals =  np.array(ffvals) / sum(ffvals) 
    
def fragment(quark, system, frame="lab", scale="p", sign=1):
            
    if frame=="lab": particle=quark
    if frame=="cm" : particle=quark.boost(system.boostvector)

    # select hadronization channel
    pid = np.random.choice(pids, size=1, p=ffractions)[0]
    mass = masses[pid]

    # select z=ph/pq and x=Eh/Eq
    e, p = particle.e, particle.p
    px, py, pz = particle.px, particle.py, particle.pz
    z = np.random.choice(zvals, size=1, p=ffvals)[0]

    # hadron momentum 
    hadron =  LorentzVector()
    if scale=="p": 
        hadron.setpxpypzm(z*px, z*py, z*pz, mass)
    if scale=="e":
        if z*e<mass: hadron.setpxpypzm(0, 0, 0, mass)
        else: 
            x = np.sqrt(z**2*e**2-mass**2)/p
            hadron.setpxpypzm(x*px, x*py, x*pz, mass)
    if frame=="cm": hadron=hadron.boost(-1.*system.boostvector)

    # return
    return [sign*pid, hadron.rapidity, hadron.pt ]

### Run Pythia

In [28]:
def generate_events_pythia(nEvent=100):
    
    # Initialize pythia 
    pythia = pythia8.Pythia();
    pythia.readString("Random:setSeed = on");
    pythia.readString("Random:seed = 0")
    
    # Process
    pythia.readString("Beams:eCM = 13000")
    pythia.readString("HardQCD:hardccbar=on");
    pythia.readString("PartonLevel:MPI=on");
    pythia.readString("Tune:pp = 14");
    
    # Stable Charm
    pythia.readString("411:maydecay=off");
    pythia.readString("421:maydecay=off");
    pythia.readString("431:maydecay=off");
    pythia.readString("4122:maydecay=off");
    pythia.readString("111:maydecay=off");
    
    #initialize
    pythia.init();
    start_time = time.time()

    # Begin event loop
    data={'lab':[], 'cm':[], 'p8':[]}
    for iEvent in range(nEvent):

        # logging info
        if (iEvent+1)% 10000 == 0: 
            print ("processed "+str(iEvent+1)+" Events after", round(time.time()-start_time,1), "seconds")
        
        # generate next 
        pythia.next();
            
        # quarks
        quark1p8 = pythia.event[5];
        quark2p8 = pythia.event[6];
        quark1=LorentzVector(quark1p8.px(), quark1p8.py(), quark1p8.pz(), quark1p8.e())
        quark2=LorentzVector(quark2p8.px(), quark2p8.py(), quark2p8.pz(), quark2p8.e())
        system = quark1+quark2
                
        # fragmentation function 
        data['lab'].append(fragment(quark1, system, frame="lab", scale="p", sign=np.sign(quark1p8.id()) ))
        data['lab'].append(fragment(quark2, system, frame="lab", scale="p", sign=np.sign(quark2p8.id()) ))
        data['cm'].append(fragment(quark1, system, frame="cm", scale="p", sign=np.sign(quark1p8.id()) ))
        data['cm'].append(fragment(quark2, system, frame="cm", scale="p", sign=np.sign(quark2p8.id()) ))
        
        # analyze events
        for iparticle, particle in enumerate(pythia.event):
            if particle.status()<0: continue
            if abs(particle.id()) in [411, 421, 431, 4122]: 
                px, py = particle.px(), particle.py()
                pz, en = particle.pz(), particle.e()
                pt, y  = np.sqrt(px**2+py**2), 0.5*np.log((en+pz)/(en-pz))
                data['p8'].append([particle.id(), y, pt])     

    
    total_xs = pythia.info.sigmaGen()*10**9
    return data, total_xs, nEvent

### Run

In [32]:
data, total_xs, nEvent = generate_events_pythia(nEvent=500000)
format_spectra([data['lab'], total_xs, nEvent], "13TeV_Pythia8_FFLab")
format_spectra([data['cm'], total_xs, nEvent], "13TeV_Pythia8_FFCM")
format_spectra([data['p8'], total_xs, nEvent], "13TeV_Pythia8_P8")

processed 10000 Events
processed 20000 Events
processed 30000 Events
processed 40000 Events
processed 50000 Events
processed 60000 Events
processed 70000 Events
processed 80000 Events
processed 90000 Events
processed 100000 Events
processed 110000 Events
processed 120000 Events
processed 130000 Events
processed 140000 Events
processed 150000 Events
processed 160000 Events
processed 170000 Events
processed 180000 Events
processed 190000 Events
processed 200000 Events
processed 210000 Events
processed 220000 Events
processed 230000 Events
processed 240000 Events
processed 250000 Events
processed 260000 Events
processed 270000 Events
processed 280000 Events
processed 290000 Events
processed 300000 Events
processed 310000 Events
processed 320000 Events
processed 330000 Events
processed 340000 Events
processed 350000 Events
processed 360000 Events
processed 370000 Events
processed 380000 Events
processed 390000 Events
processed 400000 Events
processed 410000 Events
processed 420000 Events
p